In [12]:
!pip install langchain==0.2.14
!pip install langchain-openai==0.1.8
!pip install langchainhub==0.1.20
!pip install tiktoken==0.7.0

In [17]:
import os
import getpass

# # Set OPENAI API Key

os.environ["OPENAI_API_KEY"] = getpass.getpass()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

# OR (load from .env file)

# from dotenv import load_dotenv
# load_dotenv("./.env")

The more complex version of this would use this pygithub package as shown in here:
- https://python.langchain.com/docs/integrations/toolkits/github

But let's try a simpler version that just sets up a bunch of python functions to automate different github actions like add, commit and pull requests.

Let's start with a simple commit to a branch of some repository.

First, I'll create a github repository using some simple commands.

In [14]:
# git init -b main

# git add . && git commit -m "Some commit"

# gh repo create

In [2]:
MODEL='gpt-4o-mini'
TEMP=0.0

Now, let's try asking an agent to write come code and commit the resulting code to the current branch of this repository.

To do that, let's give the agent the necessary tools it will need which in this case will be python functions that perform different github actions using the `subprocess` package.

We'll follow the basic steps for building a langchain agent:

# Steps for building a simple agent:
- Set up the LLM
- Define the tool or toolkit (list of tools)
- Set up a prompt template
- Connect llm with your tools
- Define your agent as a dict with keys: input, and agent scratchpad 
- Use the Langchain LCEL language to pipe agent, prompt the llm_with_tools variable and the output parser (which can use OpenAI function calling parser)
- Create the agent loop
- Wrap everything into the AgentExecutor
- Invoke the agent with some query input


In [18]:
# setup the llm
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=MODEL, temperature=TEMP)

Now, let's create a tool for the LLM

In [4]:
import subprocess

def github_commit_tool(commit_message="Some commit"):
    subprocess.run(["git", "add", "."])
    subprocess.run(["git", "commit", "-m", commit_message])
    subprocess.run(["git", "push", "-u", "origin", "main"])
    
    return "Committed to Github"

Now! Before we use it with langchain, let's test it by itself.

In [5]:
github_commit_tool("Testing commit for github agent demo.")

[main 079c67c] Testing commit for github agent demo.
 2 files changed, 73 insertions(+), 113 deletions(-)
 create mode 100644 notebooks/2024-08-26-12-39-39.png
branch 'main' set up to track 'origin/main'.


To https://github.com/EnkrateiaLucca/oreilly_live_training_getting_started_with_langchain.git
   3a97192..079c67c  main -> main


'Committed to Github'

Now, let's take a look at out github repository from the terminal.

![](2023-10-29-20-47-54.png)

Nice! It looks like we are good to go with this first simple tool!

Let's now make it a tool for our langchain agent by adding the @tool. 

In [19]:
from langchain.tools import tool

@tool
def github_commit_tool(commit_message="Some commit"):
    """This function uses the subprocess package to make commits to a github repo pre-defined."""
    subprocess.run(["git", "add", "."])
    subprocess.run(["git", "commit", "-m", commit_message])
    subprocess.run(["git", "push", "-u", "origin", "main"])
    
    return "Committed to Github"

tools = [github_commit_tool]

We added some documentation to our function to abide by the requirements of the tool decorator from langchain.

Now, let's test if a simple agent can use that tool!

Let's start by setting up our prompt template.

In [20]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [8]:
from langchain.agents import create_tool_calling_agent
from langchain_openai import ChatOpenAI

agent = create_tool_calling_agent(llm, tools, prompt)

In [9]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent,tools=tools, verbose=True)

In [10]:
agent_executor.invoke({"input": "Create a github commit with the message 'Testing LLM agent!'"})



> Entering new AgentExecutor chain...

Invoking: `github_commit_tool` with `{'commit_message': 'Testing LLM agent!'}`


On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


Everything up-to-date


branch 'main' set up to track 'origin/main'.
Committed to GithubThe commit with the message "Testing LLM agent!" has been successfully made to GitHub.

> Finished chain.


{'input': "Create a github commit with the message 'Testing LLM agent!'",
 'output': 'The commit with the message "Testing LLM agent!" has been successfully made to GitHub.'}

![](2024-08-26-12-39-39.png)

And voila, the agent calls the right function and creates the commit!

(ignore the github related error, let's focus on the agent capabilities).

And voila! We made a commit!

![](2023-10-30-13-04-20.png)

Ok, this is cool but, we can go beyond that, and add the ability to create a file and then commit that file to our desired repository. Let's do that!

To do it all we have to do is amplify the Agent's toolkit with another tool that can create files in the current folder.

Perfect! Now, let's add this to our github agent

In [21]:
@tool
def create_file_tool(filename, contents):
    """This function creates a file given its filename and its contents provided as inputs."""
    with open(filename, "w") as f:
        f.write(contents)
    
    return "File created"

Now, let's add that functionality to our agent by just updating the tools list.

In [22]:
tools = [github_commit_tool, create_file_tool]

In [23]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [24]:
# Let's also inspect this agent execution in langsmith!
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Let's also inspect this agent execution in langsmith!

In [26]:
agent_executor.invoke({'input': 'Create and commit a file named: "pancakes-are-the-best-breakfast.md" containing 10 bullet points on why\
    pancakes are the most delicious breakfast.'})



> Entering new AgentExecutor chain...


We can inspect this more thoroughly in the [langsmith platform](https://smith.langchain.com/).

Ok, before we finish with this example, let's add a last tool to read files from a repo, for situations where we might write some Python code and we want our agent to integrate that Python code into our repository. 

Again, we follow the same procedure:

- Create the tool
- Update our tookit
- Update our agent
- Run a test task to check if it works 

![](2023-10-30-13-42-25.png)

I think this is a great usecase for agents like these even if sometimes the performance and reliability is a bit off, because now, its a matter of optimizing the edges to get it to work for complex use cases.